In [1]:
import pandas as pd
import seaborn as sns
import nltk
from nltk.tokenize import word_tokenize
import string
from nltk.corpus import stopwords

# Agenda
<ol>

<li> Create method for basic cleaning, removing retweet tags,etc. </li>
<li> Create method for POS tagging and lemmatization </li>
<li> Run Sentiment analysis on data with basic cleaning using Vader and Affin. </li>
<li> Train multiple classifiers including Naive Bayes and SVMs on the data. </li>
<li> Write method to tag messages as neutral if average classification probability in the neighbourhood of 0.5, possibly give more weightage to Vader/Affin output </li>
<li> Setup twitter api to get the entire tweets using tweet ids </li>
<li> Setup apis for fb and reddit to be able to access vaccine related posts to enhance model. </li>

In [2]:
df_tweets = pd.read_csv(r'./data/vaccine_labeled_tweets.csv',index_col=0)

In [3]:
df_pro = pd.read_csv(r'./data/vaccine_pro.csv')
df_pro.describe()
# df_pro['message'].drop_duplicates()

,tweetid,inreplyuserid,friendscount,userid,favoritecount,followerscount,inreplytweetid,retweetcount,annotation
count,1.000000e+04,1.000000e+04,10000.000000,1.000000e+04,10000.0,1.000000e+04,1.000000e+04,10000.0,10000.0
mean,1.383379e+18,4.807362e+16,1303.354900,4.815039e+17,0.0,1.273950e+04,1.766206e+17,0.0,1.0
std,2.890019e+13,2.334947e+17,4606.207082,5.685077e+17,0.0,1.988503e+05,4.616376e+17,0.0,0.0
min,1.383327e+18,-1.000000e+00,0.000000,1.406830e+05,0.0,0.000000e+00,-1.000000e+00,0.0,1.0
25%,1.383355e+18,-1.000000e+00,192.000000,3.300326e+08,0.0,8.600000e+01,-1.000000e+00,0.0,1.0
50%,1.383378e+18,-1.000000e+00,488.000000,2.926940e+09,0.0,3.250000e+02,-1.000000e+00,0.0,1.0
75%,1.383404e+18,-1.000000e+00,1253.250000,1.094894e+18,0.0,1.279250e+03,-1.000000e+00,0.0,1.0
max,1.383427e+18,1.382914e+18,272394.000000,1.383424e+18,0.0,7.832936e+06,1.383427e+18,0.0,1.0


In [4]:
df_anti = pd.read_csv(r'./data/vaccine_anti.csv')
# df_anti['message'].drop_duplicates()

In [5]:
df_anti.columns

Index(['tweetid', 'name', 'inreplyuserid', 'createdat', 'timezone',
       'friendscount', 'accountcreated', 'location', 'userid', 'xcoordinate',
       'message', 'screenname', 'favoritecount', 'followerscount',
       'inreplytweetid', 'retweetcount', 'utcoffset', 'ycoordinate',
       'dataagent', 'place', 'inreplyscreenname', 'category', 'annotation'],
      dtype='object')

In [6]:
def simple_clean(df,msgColumn,cleanColumn):
        df[cleanColumn] = df[msgColumn].replace("^RT.*:|\s{2,}|…"," ",regex=True).str.strip()

def gen_word_tokens(message):
    words = word_tokenize(message)
    wordList = []
    for w in words:
        if w.lower() not in stop:
            wordList.append(w)
    return wordList

def clean_tweet(df,msgColumn,wordTokens):
    df[wordTokens] = df[msgColumn].apply(gen_word_tokens)

In [8]:
import string
# print(list(string.punctuation))
from nltk.corpus import stopwords
# print(stopwords.words('english'))
stop = stopwords.words('english')
stop.extend(list(string.punctuation))
print(stop)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [9]:
simple_clean(df_anti,'message','message_basicClean')
clean_tweet(df_anti,'message_basicClean','word_tokens')
df_anti

,tweetid,name,inreplyuserid,createdat,timezone,friendscount,accountcreated,location,userid,xcoordinate,...,retweetcount,utcoffset,ycoordinate,dataagent,place,inreplyscreenname,category,annotation,message_basicClean,word_tokens
0,1383427464639115272,Mohit Bhutani,-1,2021-04-17 14:29:45,None,374,Wed Sep 01 13:47:19 +0000 2010,"नई दिल्ली, भारत",185640437,None,...,0,None,None,JS Listener,None,None,vaccine,-1,"We need humility, a clear vaccine strategy &am...","[need, humility, clear, vaccine, strategy, amp..."
1,1383427463779217420,Deependra,-1,2021-04-17 14:29:44,None,85,Fri Mar 01 09:33:58 +0000 2019,None,1101415269069598720,None,...,0,None,None,JS Listener,None,None,vaccine,-1,"We need humility, a clear vaccine strategy &am...","[need, humility, clear, vaccine, strategy, amp..."
2,1383427456456019968,Jamie Pedersen,-1,2021-04-17 14:29:43,None,684,Thu Mar 24 03:22:54 +0000 2016,None,712842098228105216,None,...,0,None,None,JS Listener,None,None,vaccine,-1,Fauci admits to LYING about Covid-19 herd immu...,"[Fauci, admits, LYING, Covid-19, herd, immunit..."
3,1383427440211419136,Ram Gurjar,-1,2021-04-17 14:29:39,None,35,Mon Mar 08 03:54:44 +0000 2021,None,1368772096306778113,None,...,0,None,None,JS Listener,None,None,vaccine,-1,"We need humility, a clear vaccine strategy &am...","[need, humility, clear, vaccine, strategy, amp..."
4,1383427428874285057,Wake Up Mr West,-1,2021-04-17 14:29:36,None,160,Thu Apr 24 22:31:35 +0000 2014,None,2462304828,None,...,0,None,None,JS Listener,None,None,vaccine,-1,9/11 was an inside job Man did not land on the...,"[9/11, inside, job, Man, land, moon, Evolution..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,1382370962922995714,Victoria #NoMasksInClass,-1,2021-04-14 16:31:35,None,595,Sat May 21 13:04:08 +0000 2011,None,302596372,None,...,0,None,None,JS Listener,None,None,vaccine,-1,"Once vaccinated, you’re likely to avoid seriou...","[vaccinated, ’, likely, avoid, serious, illnes..."
9996,1382370963073998854,Phoenix Rising- Firey Mythological American Bird,-1,2021-04-14 16:31:35,None,371,Sat Dec 22 16:48:11 +0000 2018,USA,1076519780910358528,None,...,0,None,None,JS Listener,None,None,vaccine,-1,Corporate America is pushing freedom-killing v...,"[Corporate, America, pushing, freedom-killing,..."
9997,1382370965301075969,mzMolasses,-1,2021-04-14 16:31:35,None,223,Mon Feb 08 20:03:33 +0000 2010,"California, USA",112518914,None,...,0,None,None,JS Listener,None,None,vaccine,-1,When someone voluntarily tweets about the side...,"[someone, voluntarily, tweets, side, effects, ..."
9998,1382370958523183110,Steve Bishop,-1,2021-04-14 16:31:34,None,1133,Thu Oct 25 22:15:23 +0000 2018,"South West, England",1055583626564386816,None,...,0,None,None,JS Listener,None,None,vaccine,-1,"Once vaccinated, you’re likely to avoid seriou...","[vaccinated, ’, likely, avoid, serious, illnes..."


In [11]:
text = df_anti['word_tokens'][0]
pos_tag = nltk.pos_tag
pos_tag(text)

[('need', 'NN'),
 ('humility', 'NN'),
 ('clear', 'JJ'),
 ('vaccine', 'NN'),
 ('strategy', 'NN'),
 ('amp', 'JJ'),
 ('income', 'NN'),
 ('support', 'NN'),
 ('contain', 'VBP'),
 ('virus', 'NN'),
 ('related', 'VBN'),
 ('damage', 'NN'),
 ('GOI', 'NNP'),
 ('’', 'NNP'),
 ('mix', 'NN'),
 ('arrog', 'NN')]